In [3]:
#Prerequisites – Download nltk stopwords and spacy model, GENSIM
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kanimozhi.u\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#Import Packages
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # Important for visualization
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
!pip install gensim

  Using cached https://files.pythonhosted.org/packages/09/ed/b59a2edde05b7f5755ea68648487c150c7c742361e9c8733c6d4ca005020/gensim-3.8.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/0c/09/735f2786dfac9bbf39d244ce75c0313d27d4962e71e0774750dc809f2395/smart_open-1.9.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/79/e6afb3d8b0b4e96cefbdc690f741d7dd24547ff1f94240c997a26fa908d3/s3transfer-0.3.3-py2.py3-none-any.whl
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73092 sha256=76ccbbb797ccb8f8766f8d15a11d85ff93612ec9038c6708c406e2fc7316a1de
  Stored in directory: C:\Users\kanimozhi.u\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [10]:
!pip install pyLDAvis

  Using cached https://files.pythonhosted.org/packages/a5/3a/af82e070a8a96e13217c8f362f9a73e82d61ac8fff3a2561946a97f96266/pyLDAvis-2.1.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/ce/4b/6ffa76544e46614123de31574ad95758c421aae391a1764921b8a81e1eae/funcy-1.14.tar.gz
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97715 sha256=ebf41d382d4ac0dfacf1f5a01b443f2b63eea41c5cb2f4d6c92aca6bd8c3040a
  Stored in directory: C:\Users\kanimozhi.u\AppData\Local\pip\Cache\wheels\98\71\24\513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32045 sha256=b20a6530137b4245a740441ec447f8bdb056d62b8d4a170835cf8e91e40467da
  Stored in directory: C:\Users\kanimozhi.u\AppData\Local\pip\Cache\wheels\20\5a\d8\1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [8]:
!pip install spacy

  Using cached https://files.pythonhosted.org/packages/69/d8/f3103202aeca6fb0d2dbdd3a4ab1a7b86e9ad1d3cf8b23fa46bd466d64ac/spacy-2.2.3-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/d5/7e/1981d5389b75543f950026de40a9d346e2aec7e860b2800e54e65bd46c06/blis-0.4.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/3c/5a/0d1b575ed40989d74fab25723083837c220246b25f3582917135cb32453f/preshed-3.0.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/9e/ed/7edded74724747f7dfc513f85b483db7828e4a1ed072c9625188dcb633a5/thinc-7.3.1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/21/e1/e4e7b754e6be3a79c400eb766fb34924a6d278c43bb828f94233e0124a21/wasabi-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6c/f9/9a5658e2f56932e41eb264941f9a2cb7f3ce41a80cb36b2af6ab78e2f8af/catalogue-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packag

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [50]:
!pip install spacy-symspell

In [51]:
!pip install spacy_hunspell

  Created wheel for spacy-hunspell: filename=spacy_hunspell-0.1.0-cp37-none-any.whl size=3060 sha256=bccc2a05925af19db36b29695a69e813480a015020d96af0ed965d2ce95b1157
  Stored in directory: C:\Users\kanimozhi.u\AppData\Local\pip\Cache\wheels\07\b2\18\b5aa882df45e376e0f52ca7306c450f548ca150168338c0468
  Running setup.py clean for hunspell
Successfully built spacy-hunspell
Failed to build hunspell
  Running setup.py install for hunspell: started
    Running setup.py install for hunspell: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Conda\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\KANIMO~1.U\\AppData\\Local\\Temp\\pip-install-1gv25hc4\\hunspell\\setup.py'"'"'; __file__='"'"'C:\\Users\\KANIMO~1.U\\AppData\\Local\\Temp\\pip-install-1gv25hc4\\hunspell\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\KANIMO~1.U\AppData\Local\Temp\pip-wheel-7ebaji40' --python-tag cp37
       cwd: C:\Users\KANIMO~1.U\AppData\Local\Temp\pip-install-1gv25hc4\hunspell\
  Complete output (7 lines):
  C:\Conda\lib\distutils\extension.py:131: UserWarning: Unknown Extension options: 'compile_args', 'macros'
    warnings.warn(msg)
  running bdist_wheel
  running build
  running build_ext
  building 'hunspell' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Vis

In [6]:
# Import Dataset
import os

os.chdir("D:\Choogle\Data\dataset_review")
df = pd.read_csv(r'dataset2_london.csv')
reviews_datasets = df.head(20000)
#df = pd.read_json('https://raw.githubusercontent.com/datasets/master/newsgroups.json')
print(reviews_datasets.text.unique())
df.head()

["The staff in this branch of Nando's were friendly and efficient. Despite being a fairly busy Friday night the wood arrived promptly and was up to the usual standard."
 'The most tasty chicken in town with their famous Piri piri sauce. Great new restaurant, very spacious, clean. Goid service. Avoid on Arsenal match days, too many people.'
 'Really surprised and pleased, very friendly and great beer, what more could you ask, plus warm on a wet cold day'
 ...
 "This restaurant has PHENOMENAL taquitos! The rest of the food is good. Our family of four including two young kids ate from their &quot;Street Food&quot; menu - which is smaller plates including tacos, quesadillas, and the aforementioned taquitos. While the other dishes we got (include Ancho chicken tacos, pork pibril tacos, and a coupla quesadillas) was definitely good, their shrimp and crab taquito and sweet potato and feta taquito were just unbelievably good. If I could / were to go back, I think I'd just order those taquitos 

,crayon_review_id,crayon_user_id,crayon_product_id,domain,url,type,category,date_created,gid,key,...,user_id,user_name,user_location_text,user_city,user_country,user_total_reviews,user_total_reviews_range,user_helpful_reviews,user_helpful_reviews_range,partition_0
0,RR-202001000-552550429,RU-302001000-806642411,R-102001000-202456154,item_1_saturam_restaurant_review_metadata_info,https://item_1_saturam_restaurant_review_metad...,Reviews,Restaurants,2019-04-11,item_1_saturam_restaurant_review_metadata_info...,ef65c514583b9254dbd8f794c516aaca21c864076d59e3...,...,732393D49A0A1F0B63E5037F07DCDA04,Stevetarn2014,"London, United Kingdom",London,United Kingdom,423,101 to 500,1030,1001 to 5000,item_1_restaurant_review_20190609_v10_stage4_c...
1,RR-202001000-550793573,RU-302001000-802600320,R-102001000-202456154,item_1_saturam_restaurant_review_metadata_info,https://item_1_saturam_restaurant_review_metad...,Reviews,Restaurants,2019-04-11,item_1_saturam_restaurant_review_metadata_info...,e766851ea18b154b154e419691d4ee7eeb4db9d80333f4...,...,2D0CF2077ADEDA59A706B7E1E3DE26E6,T35BZcristinag,NaN,NaN,NaN,141,101 to 500,9,1 to 100,item_1_restaurant_review_20190609_v10_stage4_c...
2,RR-202001000-533398566,RU-302001000-802928560,R-102001000-200371326,item_1_saturam_restaurant_review_metadata_info,https://item_1_saturam_restaurant_review_metad...,Reviews,Restaurants,2019-04-12,item_1_saturam_restaurant_review_metadata_info...,982dced88fd267e522b3591a402d2a3f5d3cb8bc783bdd...,...,32BF09E09D5417A78AC008B76F91EF11,Q1487BW,"Bristol, United Kingdom",Bristol,United Kingdom,8,1 to 100,2,1 to 100,item_1_restaurant_review_20190609_v10_stage4_c...
3,RR-202001000-556167355,RU-302001000-803691542,R-102001000-200371326,item_1_saturam_restaurant_review_metadata_info,https://item_1_saturam_restaurant_review_metad...,Reviews,Restaurants,2019-04-12,item_1_saturam_restaurant_review_metadata_info...,ffdeac1f47ee1b9e21e3bf54869d87e1b9ab7b9a55d02f...,...,3FF79B747F3F4A0FA6BD325DA30ECDFE,Ian M,NaN,NaN,NaN,6,1 to 100,1,1 to 100,item_1_restaurant_review_20190609_v10_stage4_c...
4,RR-202001000-545446145,RU-302001000-809584645,R-102001000-200371326,item_1_saturam_restaurant_review_metadata_info,https://item_1_saturam_restaurant_review_metad...,Reviews,Restaurants,2019-04-12,item_1_saturam_restaurant_review_metadata_info...,cf0e41f130196d008f51f301712f0f48e572472e4532a1...,...,A62A1443262F2B60BCFDA5903E663B03,gillank,"Hanoi, Vietnam",Hanoi,Vietnam,140,101 to 500,41,1 to 100,item_1_restaurant_review_20190609_v10_stage4_c...


In [7]:
# Convert to list
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The staff in this branch of Nandos were friendly and efficient. Despite '
 'being a fairly busy Friday night the wood arrived promptly and was up to the '
 'usual standard.']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'staff', 'in', 'this', 'branch', 'of', 'nandos', 'were', 'friendly', 'and', 'efficient', 'despite', 'being', 'fairly', 'busy', 'friday', 'night', 'the', 'wood', 'arrived', 'promptly', 'and', 'was', 'up', 'to', 'the', 'usual', 'standard']]


In [9]:
reviews_datasets['text'][0]

"The staff in this branch of Nando's were friendly and efficient. Despite being a fairly busy Friday night the wood arrived promptly and was up to the usual standard."

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'staff', 'in', 'this', 'branch', 'of', 'nandos', 'were', 'friendly_and_efficient', 'despite_being', 'fairly', 'busy', 'friday_night', 'the', 'wood', 'arrived', 'promptly', 'and', 'was', 'up', 'to', 'the', 'usual', 'standard']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [33]:
#!python -m spacy download en_core_web_lg
!pip install -U spacy download en_core_web_lg
!pip install -U spacy download en_core_web_sm

Requirement already up-to-date: spacy in c:\conda\lib\site-packages (2.2.3)
Requirement already up-to-date: en_core_web_lg in c:\conda\lib\site-packages (2.2.5)
Requirement already up-to-date: spacy in c:\conda\lib\site-packages (2.2.3)
Requirement already up-to-date: download in c:\conda\lib\site-packages (0.3.4)
Requirement already up-to-date: en_core_web_sm in c:\conda\lib\site-packages (2.2.5)


In [34]:
!python -m spacy download en_core_web_sm


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [35]:
#import en_core_web_sm
#nlp = en_core_web_sm.load()
#doc = nlp("This is a sentence.")
#print([(w.text, w.pos_) for w in doc])

[('This', 'DET'), ('is', 'AUX'), ('a', 'DET'), ('sentence', 'NOUN'), ('.', 'PUNCT')]


In [12]:
import en_core_web_sm

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = en_core_web_sm.load()
#nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['branch', 'nando', 'friendly', 'efficient', 'fairly', 'busy', 'wood', 'arrive', 'promptly', 'usual', 'standard']]


In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]]


In [14]:
id2word[0]

'arrive'

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('arrive', 1),
  ('branch', 1),
  ('busy', 1),
  ('efficient', 1),
  ('fairly', 1),
  ('friendly', 1),
  ('nando', 1),
  ('promptly', 1),
  ('standard', 1),
  ('usual', 1),
  ('wood', 1)]]

In [16]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.037*"menu" + 0.028*"try" + 0.028*"busy" + 0.020*"serve" + 0.019*"bit" + '
  '0.017*"get" + 0.016*"bar" + 0.016*"choice" + 0.016*"go" + 0.015*"mexican"'),
 (1,
  '0.038*"big" + 0.034*"cheese" + 0.029*"french" + 0.028*"money" + 0.019*"old" '
  '+ 0.019*"kid" + 0.018*"short" + 0.018*"mix" + 0.018*"plate" + '
  '0.017*"extra"'),
 (2,
  '0.049*"table" + 0.049*"order" + 0.034*"would" + 0.029*"come" + 0.029*"wait" '
  '+ 0.025*"time" + 0.020*"take" + 0.019*"drink" + 0.018*"arrive" + '
  '0.017*"could"'),
 (3,
  '0.064*"restaurant" + 0.028*"make" + 0.023*"experience" + 0.022*"quite" + '
  '0.020*"find" + 0.018*"go" + 0.017*"fresh" + 0.017*"quick" + 0.016*"staff" + '
  '0.014*"way"'),
 (4,
  '0.058*"dinner" + 0.050*"chip" + 0.034*"cook" + 0.034*"server" + '
  '0.034*"chain" + 0.030*"favourite" + 0.018*"vegetarian" + 0.017*"prepare" + '
  '0.015*"mostly" + 0.012*"move"'),
 (5,
  '0.086*"food" + 0.059*"good" + 0.046*"great" + 0.032*"place" + '
  '0.027*"service" + 0.023*"staff" + 0.020*

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.685872040043937

Coherence Score:  0.3928124692665542


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Conda\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.263140  0.121956       1        1  27.352657
2     -0.161633 -0.320986       2        1  18.811193
0     -0.202926 -0.068122       3        1  14.732854
3     -0.128686  0.170061       4        1  13.502413
6      0.092106  0.041494       5        1   8.112405
7      0.002801  0.151583       6        1   7.473368
1      0.167028 -0.041901       7        1   3.555397
4      0.175881 -0.018664       8        1   3.098179
9      0.170621 -0.020029       9        1   2.742150
8      0.147947 -0.015392      10        1   0.619377, topic_info=     Category         Freq        Term        Total  loglift  logprob
55    Default  1319.000000        food  1319.000000  30.0000  30.0000
56    Default   835.000000        good   835.000000  29.0000  29.0000
18    Default   498.000000  restaurant   498.000000  28.0000  28.0000
275   Default   477.000000       order   477.000000  27.0000  27.0000
27    Default   658.000000       great   658.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
4057  Topic10     1.104509  intolerant     1.988858   4.4961  -5.6732
204   Topic10     1.041700      stroll     1.925468   4.4699  -5.7317
205   Topic10     0.982092         sun     1.865720   4.4425  -5.7906
2770  Topic10     0.981086     wedding     1.864773   4.4420  -5.7916
1692  Topic10     0.953001     ideally     1.836627   4.4281  -5.8207

[429 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1158      4  0.968840             able
833       3  0.413975         actually
833       5  0.575966         actually
482       9  0.982086              add
2081     10  0.836680        adventure
...     ...       ...              ...
228       4  0.993629  would_recommend
692       4  0.976991             year
1595      8  0.964027        yesterday
164       7  0.361993            young
164       8  0.615388            young

[541 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 1, 4, 7, 8, 2, 5, 10, 9])